In [ ]:
from io import BytesIO
import imageio
import rasterio
from rasterio.io import MemoryFile
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import json
import os

In [ ]:
def plot_geojson(gjs, *args, **kwargs):
    points = np.array(gjs['coordinates'][0])
    return plt.plot(points[:,0], points[:,1], *args, **kwargs)

In [ ]:
from rgdc import Rgdc

client = Rgdc(username='username', password='password')

## Search

In [ ]:
bbox = {"type":"Polygon",
        "coordinates":[[[-105.45091240368326,39.626245373878696],
                                         [-105.45091240368326,39.929904289147274],
                                         [-104.88775649170178,39.929904289147274],
                                         [-104.88775649170178,39.626245373878696],
                                         [-105.45091240368326,39.626245373878696]]]}

q = client.search(query=json.dumps(bbox), predicate='intersects', datatype='raster')
len(q)

In [ ]:
for s in q:
    print(s['subentry_name'])

In [ ]:
plot_geojson(bbox, 'k--', label='Search Region')
for s in q:
    plot_geojson(s['footprint'], label=s['subentry_name'])
plt.legend()
plt.title(f'Count: {len(q)}')

## Inspect Raster

In [ ]:
raster = client.session.get(q[0]['detail']).json()
raster

In [ ]:
plot_geojson(bbox, 'k--')
plot_geojson(raster['outline'], 'r')

In [ ]:
count = len(raster['parent_raster']['image_set']['images'])
count

In [ ]:
load_image = lambda imbytes: imageio.imread(BytesIO(imbytes))

for i in range(count):
    thumb_bytes = client.download_raster_entry_thumbnail(raster['id'], band=i)
    thumb = load_image(thumb_bytes)
    plt.subplot(1, count, i+1)
    plt.imshow(thumb)
plt.tight_layout()
plt.show()

## Download Raster

In [ ]:
paths = client.download_raster_entry(raster['id'])
paths

In [ ]:
rasters = [rasterio.open(im) for im in paths['images']]

In [ ]:
for i, src in enumerate(rasters):
    plt.subplot(1, count, i+1)
    ax = plt.gca()
    show(src, ax=ax)
plt.tight_layout()
plt.show()